In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

pd.set_option("display.max_rows", 2000)

In [2]:
def find_bottles(bottles):

    bottle_ids = bottles.find_all("h6")                                      
    bottle_names = bottles.find_all("h4", text=re.compile("Macallan.*")) # So that im just getting the bottles and no other tags
    
    all_bottles_prices = bottles.find_all(class_= ["sold", "bndone", "openprice", "unsold"]) # So that we have all bottles, sold and not sold. 
    
    put_together = list(zip(bottle_ids, bottle_names, all_bottles_prices)) # putting all the info into a list
    
    
    df = pd.DataFrame(put_together, columns=["Lot number", "Name", "Price"])
    for i in df.columns:
        df[i] = df[i].apply(lambda x: BeautifulSoup(str(x), "lxml").get_text())
    #df = df.loc[~df["Price"].str.contains("This lot is no longer available")]  # Getting just sold bottles WILL NEED MOVING ELSEWHERE LATER

    return df

#bottles = soup
#df = find_bottles(bottles)
#df.head(5)



In [3]:
link2 = "https://www.scotchwhiskyauctions.com/auctions/" 
webpage_response2 = requests.get(link2)

webpage2 = webpage_response2.content
soup2 = BeautifulSoup(webpage2, "lxml")

auction_links = [i["href"] for i in soup2.find_all("a", class_="auction", href=True)] # Getting the auction links
auction_links.reverse()
x = auction_links[79:85]
print(auction_links)

['auctions/14-the-inaugural-auction/', 'auctions/15-the-second-auction/', 'auctions/18-the-third-auction/', 'auctions/23-the-fourth-auction/', 'auctions/27-the-fifth-auction/', 'auctions/29-the-sixth-auction/', 'auctions/30-the-seventh-auction/', 'auctions/31-the-eighth-auction/', 'auctions/33-the-ninth-auction/', 'auctions/34-the-10th-auction/', 'auctions/37-the-11th-auction/', 'auctions/38-the-12th-auction/', 'auctions/39-the-13th-auction/', 'auctions/40-the-14th-auction/', 'auctions/43-the-15th-auction/', 'auctions/44-the-16th-auction/', 'auctions/45-the-17th-auction/', 'auctions/46-the-18th-auction/', 'auctions/47-the-19th-auction/', 'auctions/48-the-20th-auction/', 'auctions/49-the-21st-auction/', 'auctions/50-the-22nd-auction/', 'auctions/51-the-23rd-auction/', 'auctions/53-the-24th-auction/', 'auctions/55-the-25th-auction/', 'auctions/56-the-26th-auction/', 'auctions/58-the-27th-auction/', 'auctions/59-the-28th-auction/', 'auctions/60-the-29th-auction/', 'auctions/61-the-30th-au

In [ ]:
#link = "https://www.scotchwhiskyauctions.com/auctions/181-the-137th-auction/?page=1"
#webpage_response =requests.get(link)
#
#webpage = webpage_response.content
#soup = BeautifulSoup(webpage, "lxml")

df = pd.DataFrame(columns=["Lot number", "Name", "Price"])
count = 0
auction_count = 0
if __name__ == "__main__":
    # Iterating through all the auctions
    for auction in auction_links:#NEED TO CHANGE BACK
        print("-----") # Make the terminal a bit easier to read
        auction_count += 1
        print(f"Auction number: {auction_count}")
        
        webpage_response = requests.get("https://www.scotchwhiskyauctions.com/" + auction + "?q=macallan&search=c") #   Getting a auction page

        webpage = webpage_response.content
        soup = BeautifulSoup(webpage, "lxml")

        # Getting the number of pages for each auction
        number_pages_tag = soup.find_all("option")
        total_auction_pages_list = [i.text for i in number_pages_tag][-3].split() # Getting the pages string, and putting in a list to get the last value
        
        number_of_pages = total_auction_pages_list[-1] # Getting the pages last value (being the total number of pages on that auction, or "page" if there was only one page)
        print(number_of_pages)
        
        # The reason why we have a try and except clause is to account for the fact that some pages only have one page, so if we arent returned a number of pages we want to just scrape once, which is what the except is doing. 
        try: 
            for page in range(1, int(number_of_pages)+1): # Has to be +1 as we have zero indexing. 
                link = "https://www.scotchwhiskyauctions.com/"
                link = link + auction + "?q=macallan&search=c" + "&page=" + str(page)
                webpage_response = requests.get(link)


                webpage = webpage_response.content
                soup = BeautifulSoup(webpage, "lxml")

                current_page = find_bottles(soup)
                df = pd.concat([df, current_page], ignore_index=True)

                time.sleep(1) 
                count += 1
                print(f"Scraped: {count} pages")
        except:
            link = "https://www.scotchwhiskyauctions.com/"

            # In this one we dont have the page number
            link = link + auction + "?q=macallan&search=c"    
            webpage_response = requests.get(link)
            webpage = webpage_response.content
            soup = BeautifulSoup(webpage, "lxml")

            current_page = find_bottles(soup)
            df = pd.concat([df, current_page], ignore_index=True)
            
            time.sleep(1) 
            count += 1
            print(f"Scraped: {count} pages")
        

print(df)

#df.to_csv("Macallan_whisky.csv")
